Account Talk : Program to feed Account data to ChatGpt and allow user to interact with the data

In [ ]:
 !pip install openai pandas tiktoken

In [ ]:
import os
import openai
import pandas as pd
import tiktoken
import json


openai.api_key = ""
openai.organization = ""

Load the account data

In [ ]:
df = pd.read_csv('data_cat_all.csv',nrows=90)
json_data = df["all"].to_json(orient='records')

The api operates for chatgpt operates on a token system.
This routine calculates the number of tokens so we can not exceed that amount. 

In [ ]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
   
        try:
            encoding = tiktoken.encoding_for_model(model)
        except KeyError:
            encoding = tiktoken.get_encoding("cl100k_base")
        if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
            num_tokens = 0
            for message in messages:
                num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
                for key, value in message.items():
                    num_tokens += len(encoding.encode(value))
                    if key == "name":  # if there's a name, the role is omitted
                        num_tokens += -1  # role is always required and always 1 token
            num_tokens += 2  # every reply is primed with <im_start>assistant
            return num_tokens
        else:
            raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
    See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")



This is where we create the prompt. The prompt system for the new chat gpt-3.5-turbo-0301 can be found at this link. 
https://github.com/openai/openai-cookbook/blob/main/examples/How_to_format_inputs_to_ChatGPT_models.ipynb


In [ ]:
conversation=[{"role": "system", "content": "You are a helpful accountant who provides detailed analysis of transactions"}]
conversation.append({"role": "system", "content": json_data})

The following code will call openai using the API Key , it creates a input prompt. Once the user fills out the prompt it will then send the request. 
TODO: The token system currently checks the conversation object and if it is over 3500 (a little less than the 4096 available) it prunes the conversation down to just the core data . 
        This can cause it to lose the context of the conversation because it will prune all previous requests for example

        User: how much did I spend on Amazon? 
        AI: You spent x.xxx on amazon
        User: Please show me the transactions (which should be related to the previous request)
        AI: <SHOWS ALL TRANSACTIONS, NOT AMAZON TRANSACTIONS>

        This because the context got pruned between the first and second user request.. not sure how to fix.. open to ideas. 

this code will also create a file called chatgpt_responses.txt that will include all of your calls and requests in case you want to repeat them. 

In [ ]:
with open("chatgpt_responses.txt", "w") as f:
    
    while(True):
        user_input = input("")     

        tokens = num_tokens_from_messages(conversation)
        if tokens > 3500:
            #conversation = [line for line in conversation if line['role'] != 'user']
            conversation = conversation[:2]
            print("Tokens exceeded 4096")
            #print(conversation)
        conversation.append({"role": "user", "content": user_input})
        print(f"{num_tokens_from_messages(conversation)} prompt tokens counted.")



        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages = conversation,
            temperature=1,
            max_tokens=250,
            top_p=0.9
        )

        #conversation.append({"role": "assistant", "content": response['choices'][0]['message']['content']})
        print("\n" + response['choices'][0]['message']['content'] + "\n")
        print(f'{response["usage"]["prompt_tokens"]} prompt tokens used.')
    
       # Save response to file
        f.write("User: " + user_input + "\n")
        f.write("AI: " + response['choices'][0]['message']['content'] + "\n\n")